<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dara Loading and Preprocessing</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conclustions</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# 01 - ***ML for Texts*** - Project for WikiShop

[Nice reading about BERT](https://docs.deeppavlov.ai/en/master/features/models/bert.html)

BERT (Bidirectional Encoder Representations from Transformers) is a Transformer pre-trained on masked language model and next sentence prediction tasks. This approach showed state-of-the-art results on a wide range of NLP tasks in English.

### Project Goal
The project goal is a binary text classification using BERT model.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [107]:
!pip3 install -q transformers


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [2]:
import sys
!{sys.executable} -m pip install spacy -q
!{sys.executable} -m spacy download en_core_web_sm -q

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/pip/__main__.py", line 9, in <module>
    if sys.path[0] in ("", os.getcwd()):
PermissionError: [Errno 1] Operation not permitted
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
2022-11-30 20:54:17.459138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to us

In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
import re
import spacy
from tqdm import notebook
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,make_scorer
from catboost import CatBoostClassifier
import torch
import transformers
from tqdm import notebook

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

import pandas as pd
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None 

## 1 Подготовка

In [9]:
data = pd.read_csv('/Users/yuliabezginova/PycharmProjects/00_files-for_NLP/toxic_comments.csv', index_col=[0])

In [10]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [11]:
data.toxic.unique()

array([0, 1])

In [12]:
# проверка на пропуски в таргете
data['toxic'].isna().sum()

0

### 1.1 Работаем со стоп-словами

In [13]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliabezginova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.3 Переведём тексты в стандартный для Python формат: кодировку Unicode U.

In [14]:
# Изменим кодировку методом astype():
corpus = data['text'].values.astype('U')
# corpus_train = X_train.values.astype('U')
# corpus_test = X_test.values.astype('U')

### 1.4 Создадим функцию для лемматизации и очистки текста

In [15]:
def lemmatize(text):
    m = Mystem()
    m = re.sub(r'[^a-zA-Z ]', ' ', text)
    # объединим элементы в одну строку пробелом или без него
    lem = ' '.join(m.split())
    return lem

#### Протестируем работу функции лемматизации и очистки текста.

In [16]:
print("Исходный текст:", corpus[181])
print("Очищенный и лемматизированный текст:", lemmatize(corpus[181]))

Исходный текст: you are a stupid fuck 

and your mother's cunt stinks
Очищенный и лемматизированный текст: you are a stupid fuck and your mother s cunt stinks


#### Применим функцию для лемматизации и очистки текста

In [17]:
data['lem_text'] = data.text.apply(lemmatize)

In [18]:
data.head()

,text,toxic,lem_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### 1.5 Делим данные на тестовую и обучающую выборки

In [19]:
features = data.drop(columns='toxic', axis=1)
target = data['toxic']

In [20]:
features.shape

(159292, 2)

In [21]:
target.shape

(159292,)

In [22]:
# Разделяю на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    test_size=0.2, 
                                                    random_state=5)

In [23]:
print('Train features sample size:', X_train.shape[0])
print('Train target sample size:', y_train.shape[0])
print()
print('Test features sample size:', X_test.shape[0])
print('Test target sample size:', y_test.shape[0])
print()
print('Test sample, %:', round(X_test.shape[0] / (X_test.shape[0] + X_train.shape[0]) * 100, 2))

Train features sample size: 127433
Train target sample size: 127433

Test features sample size: 31859
Test target sample size: 31859

Test sample, %: 20.0


### ВЫВОД: Исходные данные разделены на тестовые и обучающие, выделены признаки и таргетные значения. Дальнейшие преобразования текста будем производить только с признаками (текстом).

Применим модель "мешка слов" к набору данных.

### 1.6 Модель «мешка слов» реализована в классе CountVectorizer

Мы импортируем класс CountVectorizer, создам экземпляр класса и подгоняем модель к нашим данным для анализа тональности твитов.

In [24]:
vect = CountVectorizer(stop_words=stopwords)

In [25]:
# Считаю Tf-idf для обучающей
X_train_vect = vect.fit_transform(X_train.lem_text)

# Считаю Tf-idf для тестовой
X_test_vect = vect.transform(X_test.lem_text)

Перед тем как мы пытаемся улучшить выделение признаков, давайте измерим качество модели, построив классификатор. 

У нас есть обучающие метки, хранящиеся в y_train и обучающие данные, представленные в виде «мешка слов» X_train, таким образом, мы можем обучить классификатор по этим данным. 

Как правило, для подобных высокоразмерных разреженных данных лучше всего работают линейные модели типа LogisticRegression.

### 1.7 Масштабирование признаков с помощью TF-IDF

Следующий подход вместо исключения несущественных признаков пытается масштабировать признаки в зависимости от степени их информативности. Одним из наиболее распространенных способов такого масштабирования является метод частота термина-обратная частота документа (term frequency-inverse document frequency, tf-idf). 

**Идея этого метода заключается в том, чтобы присвоить большой вес термину, который часто встречается в конкретном документе, но при этом редко встречается в остальных документах корпуса. Если слово часто появляется в конкретном документе, но при этом редко встречается в остальных документах, оно, вероятно, будет описывать содержимое этого документа лучше.**

Мешок слов учитывает частоту употребления слов. Посмотрим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте.

Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, «частота терма, или слова»; inverse document frequency, «обратная частота документа, или текста»). То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.

#### TFIDF = TF * IDF

IDF нужна в формуле, чтобы уменьшить вес слов, наиболее
распространённых в любом другом тексте заданного корпуса.
IDF зависит от общего числа текстов в корпусе (D) и количества
текстов, в которых это слово встречается (d).

Большая величина TF-IDF говорит об уникальности слова в тексте
по отношению к корпусу. Чем чаще оно встречается в конкретном
тексте и реже в остальных, тем выше значение TF-IDF.

**Если данные разделены на обучающую и тестовую выборки,
функцию fit() запустим только на обучающей. Иначе тестирование
будет нечестным: в модели будут учтены частоты слов из тестовой
выборки.**

Сначала переведем выборки X_train, X_test в векторный вид.


In [26]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [27]:
# Считаю Tf-idf для обучающей
tf_idf_X_train = count_tf_idf.fit_transform(X_train['lem_text'])

# Считаю Tf-idf для тестовой
tf_idf_X_test = count_tf_idf.transform(X_test['lem_text'])

In [28]:
print("Размер матрицы TF-IDF X_train:", tf_idf_X_train.shape)
print("Размер матрицы TF-IDF X_test:", tf_idf_X_test.shape)

Размер матрицы TF-IDF X_train: (127433, 147485)
Размер матрицы TF-IDF X_test: (31859, 147485)


### ВЫВОД: Данные очищены от странных букв и знаков, знаки пунктуации и стоп-слова удалены, данные разделены на тестовую и обучающую выборки, в каждой из которой выделены признаки и целевое значение. Выборки признаков приведены к векторному виду и готовы к обучению.

## 2 Обучение

### 2.1 Logistic Regression с использованием перекрестной проверки для модели «мешка слов» в классе CountVectorizer().

In [29]:
logreg = LogisticRegression(random_state=5)

In [30]:
param_grid = [
    {'C': [0.01, 0.1, 1, 2, 10, 100], 
     'penalty': ['l1', 'l2']}
]

In [31]:
logreg_grid = GridSearchCV(logreg, 
                           param_grid, 
                           cv=3, 
                           verbose=False, 
                           n_jobs=-1)

logreg_grid.fit(X_train_vect, y_train)

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=5), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 2, 10, 100],
                          'penalty': ['l1', 'l2']}],
             verbose=False)

In [32]:
print("Наилучшее значение перекрестной проверки: {:.2f}".format(logreg_grid.best_score_))
print("Наилучшие параметры: ", logreg_grid.best_params_)

Наилучшее значение перекрестной проверки: 0.96
Наилучшие параметры:  {'C': 2, 'penalty': 'l2'}


Теперь мы можем оценить на тестовом наборе обобщающую способность при использовании данной настройки параметра:

In [33]:
print("Правильность на тестовом наборе: {:.2f}".format(logreg_grid.score(X_test_vect, y_test)))

Правильность на тестовом наборе: 0.96


In [35]:
y_pred_vect = logreg_grid.predict(tf_idf_X_test)

In [36]:
confusion_matrix(y_pred_vect, y_test)

array([[28565,  2220],
       [    2,  1072]])

### 2.2 Logistic Regression с использованием решетчатого поиска для модели «мешка слов» в классе TfidfVectorizer()

Поскольку tf-idf фактически использует статистические свойства
обучающих данных, мы воспользуемся конвейером, чтобы убедиться в достоверности результатов решетчатого поиска.

In [164]:
logreg = LogisticRegression(random_state=5)

In [37]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(logreg, param_grid, cv=3)

grid.fit(tf_idf_X_train, y_train)

print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

Наилучшее значение перекрестной проверки: 0.96


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
y_pred_tfidf = grid.predict(tf_idf_X_test)

In [41]:
confusion_matrix(y_pred_tfidf, y_test)

array([[28295,  1027],
       [  272,  2265]])

### 2.3 CatBoost в модели "мешка слова" в классе TfidfVectorizer()

In [ ]:
# initiatint the model
catboost = CatBoostClassifier(random_seed=5,
                              loss_function='Logloss')

# preparing parameters to ask GridSearch, which contains cross validation
parameters_cat = {'iterations':[50], 
                  'learning_rate': np.arange(0.1,1,0.2)}

gsearch_catboost = GridSearchCV(catboost, 
                                param_grid=parameters_cat,
                                verbose=False)

gsearch_catboost.fit(tf_idf_X_train, y_train)

0:	learn: 0.5939473	total: 3.42s	remaining: 2m 47s
1:	learn: 0.5154453	total: 4.94s	remaining: 1m 58s
2:	learn: 0.4532370	total: 6.17s	remaining: 1m 36s
3:	learn: 0.4068591	total: 7.6s	remaining: 1m 27s
4:	learn: 0.3720388	total: 8.98s	remaining: 1m 20s
5:	learn: 0.3437147	total: 10.2s	remaining: 1m 14s
6:	learn: 0.3206467	total: 11.6s	remaining: 1m 11s
7:	learn: 0.3029510	total: 12.7s	remaining: 1m 6s
8:	learn: 0.2886673	total: 14s	remaining: 1m 3s
9:	learn: 0.2760725	total: 15.3s	remaining: 1m 1s
10:	learn: 0.2669043	total: 16.5s	remaining: 58.5s
11:	learn: 0.2594292	total: 17.7s	remaining: 56.2s
12:	learn: 0.2533731	total: 19.1s	remaining: 54.3s
13:	learn: 0.2484205	total: 20.3s	remaining: 52.3s
14:	learn: 0.2440143	total: 21.9s	remaining: 51s
15:	learn: 0.2400490	total: 23.2s	remaining: 49.2s
16:	learn: 0.2370516	total: 24.4s	remaining: 47.4s
17:	learn: 0.2336507	total: 25.8s	remaining: 45.8s
18:	learn: 0.2309671	total: 26.9s	remaining: 44s
19:	learn: 0.2285956	total: 28.1s	remaini

13:	learn: 0.2479260	total: 21.2s	remaining: 54.6s
14:	learn: 0.2434892	total: 22.4s	remaining: 52.3s
15:	learn: 0.2389627	total: 23.7s	remaining: 50.4s
16:	learn: 0.2354247	total: 25s	remaining: 48.5s
17:	learn: 0.2322560	total: 26.1s	remaining: 46.4s
18:	learn: 0.2298084	total: 27.2s	remaining: 44.4s
19:	learn: 0.2273066	total: 28.4s	remaining: 42.6s
20:	learn: 0.2247383	total: 29.8s	remaining: 41.1s
21:	learn: 0.2224923	total: 31s	remaining: 39.4s
22:	learn: 0.2206985	total: 32.2s	remaining: 37.8s
23:	learn: 0.2193029	total: 33.4s	remaining: 36.2s
24:	learn: 0.2178802	total: 34.6s	remaining: 34.6s
25:	learn: 0.2164012	total: 35.9s	remaining: 33.1s
26:	learn: 0.2150215	total: 37.1s	remaining: 31.6s
27:	learn: 0.2134995	total: 38.2s	remaining: 30s
28:	learn: 0.2123366	total: 39.4s	remaining: 28.5s
29:	learn: 0.2110873	total: 40.8s	remaining: 27.2s
30:	learn: 0.2098515	total: 42.1s	remaining: 25.8s
31:	learn: 0.2085974	total: 43.3s	remaining: 24.4s
32:	learn: 0.2072060	total: 44.5s	rem

### 2.4 Gradient Boosting в модели "мешка слова" в классе TfidfVectorizer()

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [ ]:
model = lgb.LGBMClassifier(random_state=5)

In [ ]:
params_LGBMClassifier = {    
    
    'n_estimators': [50, 100], 
    'learning_rate': [0.05, 0.1], 
    'verbose' : [-1],
#     'boosting_type' : ['gbdt'],
#     'num_leaves': [100],
    'max_depth' : [-1],
#     'min_split_gain' : [0.0],
#     'min_child_samples' : [20],
#     'subsample' : [1.0],
#     'subsample_freq' : [0],
#     'colsample_bytree': [1.0],
#     'reg_alpha' : [0.0],
#     'reg_lambda' : [0.0],
#     'random_state' : [5],
#     'silent' : [True]
}

LGBMClassifier_gsearch = GridSearchCV(estimator=model, 
                                     cv=3, 
                                     param_grid=params_LGBMClassifier)

LGBMClassifier_gsearch.fit(tf_idf_X_train, y_train)

### 2.5 Сравним модели

In [ ]:
# Logistic Regression CountVectorizer
best_score_vect = round(logreg_grid.best_score_, 2)
print("Best score Logistic Regression CountVectorizer: {:.2f}".format(best_score_vect))

In [173]:
# Logistic Regression TF-IDF
best_score_tfidf = round(grid.best_score_, 2)
print('Best score Logistic Regression (TF-IDF): {:.2f}'.format(best_score_tfidf))

Best score Logistic Regression (TF-IDF): 0.96


In [42]:
# CatBoost
best_score_catboost = round(gsearch_catboost.best_score_, 2)
print('Best score CatBoostClassifier (TF-IDF): {}'.format(best_score_catboost))

NameError: name 'gsearch_catboost' is not defined

In [175]:
# LGBMClassifier
best_score_LGBM = round(LGBMClassifier_gsearch.best_score_, 2)
print('Best score LGBMClassifier (TF-IDF): {}'.format(best_score_LGBM))

NameError: name 'LGBMClassifier_gsearch' is not defined

In [ ]:
# Generate the best score plot

plt.rcParams.update({'font.size': 19, 
                     'text.color' : 'white', 
                     'axes.labelcolor' : "blue"})
# plt.rcParams.update({'axes.titlesize': 'large'})
fig, ax = plt.subplots(figsize=(19, 6))

x = ['Logistic Regression (Count)', 
     'Logistic Regression (TF-IDF)', 
     'Cat Boost',
    'Light GBM']
y = [best_score_vect, 
     best_score_tfidf, 
     best_score_catboost,
     best_score_LGBM]

ax.bar(x, y, width=0.4, color='#008B8B')
ax.set_title('best_score_ for each ML algorithm', fontsize=18)
ax.set_xlabel('Regression Models', fontsize=18)
ax.set_ylabel('best_score_', fontsize=18)
ax.set_ylim(-45, 5)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 0.05, s=str(value), ha='center')
    
plt.tight_layout();

## 3 Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны